In [3]:
import cv2
from astropy.io import fits
from glob import glob
import os
import numpy as np

# Ruta de la carpeta con los archivos FITS
folder_fits = r"D:\Tecnicas_observacionales\Ocultacion"

# Ruta de la carpeta de salida para el video
output_folder = r"D:\Tecnicas_observacionales\Ocultacion\Output"

# Asegurarse de que la carpeta de salida existe
os.makedirs(output_folder, exist_ok=True)

# Nombre del archivo de salida
output_filename = "Moon_1.mp4"

# Ruta completa del archivo de video de salida
output_path = os.path.join(output_folder, output_filename)

# Obtener la lista de archivos FITS en la carpeta
fits_files = glob(os.path.join(folder_fits, "*.fits"))

# Variables para el video
video = None
height, width = None, None
i = 0
# Iterar sobre cada archivo FITS
for fits_file in fits_files:
    # Abrir el archivo FITS
    hdul = fits.open(fits_file)

    # Calcular el porcentaje completado
    perc = (i / len(fits_files)) * 100
    
    print(f"Procesando frames: {perc}% completado")
    print("-------------------------------")

    # Incrementar el contador
    i += 1

    # Obtener los datos de la imagen FITS
    fits_data = hdul[0].data.T

    # Escalar los valores de píxeles a 0-255
    fits_data = (
        (fits_data - np.min(fits_data)) / (np.max(fits_data) - np.min(fits_data)) * 255
    )

    # Convertir la imagen FITS a formato de imagen OpenCV (uint8)
    fits_data_uint8 = np.uint8(fits_data)

    # Obtener las dimensiones de la imagen
    height, width = fits_data_uint8.shape

    # Inicializar el video una vez que se obtengan las dimensiones
    if video is None:
        video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"DIVX"), 4, (width, height))

    # Añadir la imagen al video
    video.write(cv2.cvtColor(fits_data_uint8, cv2.COLOR_GRAY2BGR))

# Liberar el objeto VideoWriter
if video is not None:
    video.release()

print(f"Video guardado en: {output_path}")

Procesando frames: 0.0% completado
-------------------------------
Procesando frames: 0.38314176245210724% completado
-------------------------------
Procesando frames: 0.7662835249042145% completado
-------------------------------
Procesando frames: 1.1494252873563218% completado
-------------------------------
Procesando frames: 1.532567049808429% completado
-------------------------------
Procesando frames: 1.9157088122605364% completado
-------------------------------
Procesando frames: 2.2988505747126435% completado
-------------------------------
Procesando frames: 2.681992337164751% completado
-------------------------------
Procesando frames: 3.065134099616858% completado
-------------------------------
Procesando frames: 3.4482758620689653% completado
-------------------------------
Procesando frames: 3.8314176245210727% completado
-------------------------------
Procesando frames: 4.21455938697318% completado
-------------------------------
Procesando frames: 4.59770114942528

In [13]:
len(fits_files)

1863

In [ ]:
import cv2
import numpy as np
from scipy.stats import norm
import os
import matplotlib.pyplot as plt

# Parámetros
frame_size = (500, 500)  # Tamaño de cada frame (alto, ancho)
num_frames = 20  # Número de frames a generar
oscillation_range = 3  # Rango de oscilación de los puntos blancos
halo_size = 2  # Tamaño del halo
des = 60  # Desplazamiento de los puntos

# Inicializar la posición inicial de los puntos blancos
x1, y1 = frame_size[0] // 2, frame_size[1] // 2  # Centro del frame
x2, y2 = frame_size[0] // 2 + des, frame_size[1] // 2 + des  # Un poco desplazado

# Lista para almacenar las distancias
distancias = []

# Crear y guardar cada frame
output_dir = "D:/GitHub/Tecnicas/Seeing/data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in range(num_frames):
    # Crear una imagen negra
    frame = np.zeros((frame_size[0], frame_size[1]), dtype=np.uint8)

    # Oscilar los puntos blancos
    x1_new = x1 + int(oscillation_range * np.sin(i * 0.2))
    y1_new = y1 + int(oscillation_range * np.cos(i * 0.2))
    x2_new = x2 + int(oscillation_range * np.cos(i * 0.2))
    y2_new = y2 + int(oscillation_range * np.sin(i * 0.2))

    # Calcular la distancia entre los puntos
    dist = np.sqrt((x2_new - x1_new) ** 2 + (y2_new - y1_new) ** 2)
    distancias.append(dist)  # Guardar la distancia en la lista

    # Asegurar que los puntos blancos se mantengan dentro de los límites del frame
    x1_new = np.clip(x1_new, 1, frame_size[1] - 2)
    y1_new = np.clip(y1_new, 1, frame_size[0] - 2)
    x2_new = np.clip(x2_new, 1, frame_size[1] - 2)
    y2_new = np.clip(y2_new, 1, frame_size[0] - 2)

    # Función para dibujar el punto con "halo"
    def draw_star(frame, x, y, halo_size):
        frame[y, x] = 255
        for i in range(1, halo_size + 1):
            for dx in range(-i, i + 1):
                for dy in range(-i, i + 1):
                    if 0 <= x + dx < frame_size[1] and 0 <= y + dy < frame_size[0]:
                        if abs(dx) == i or abs(dy) == i:
                            frame[y + dy, x + dx] = 180

    # Dibujar los puntos blancos con halo en el frame
    draw_star(frame, x1_new, y1_new, halo_size)
    draw_star(frame, x2_new, y2_new, halo_size)

    # Agregar la distancia como texto en el frame
    text = f"Distancia: {dist:.2f} px"
    # cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, 2)

    # Mostrar el frame
    cv2.imshow("Oscilacion de Puntos", frame)

    # Guardar el frame en la ruta especificada
    frame_path = os.path.join(output_dir, f"frame_{i:03d}.jpg")
    cv2.imwrite(frame_path, frame)

    # Esperar un momento para visualizar los cambios
    if cv2.waitKey(100) & 0xFF == ord("q"):
        break

# Cerrar ventanas de OpenCV
cv2.destroyAllWindows()

# Calcular media y desviación estándar
mu = np.mean(distancias)
std = np.std(distancias)

# Generar el histograma de distancias
weights = np.ones_like(distancias) / len(distancias)
plt.hist(distancias, weights=weights, bins=5)

# Graficar la función de densidad de probabilidad (PDF)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, "k", linewidth=2)

# Título y etiquetas
plt.title(r"$\mathrm{Histograma\ de\ Distancias:}\ \mu=%.2f,\ \sigma=%.2f$" % (mu, std))
plt.xlabel("Distancia (px)")
plt.ylabel("Frecuencia")
plt.show()